In [11]:
import cv2
import numpy as np


# **Questão 1. Implementação da Câmera Lenta.**

In [ ]:
video_path = 'onca.mp4' 
fator = 8 

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Erro: Não foi possível abrir o vídeo em {video_path}")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Vídeo de entrada: {video_path}, FPS: {fps}, Dimensões: {width}x{height}")


Vídeo de entrada: onca.mp4, FPS: 29.97002997002997, Dimensões: 1280x720


In [ ]:
outrep_width = width
outrep_height = height

outlin_width = width
outlin_height = height

outopt_width = width
outopt_height = height

outcomb_width = 3 * width 
outcomb_height = height

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
outrep_path = 'imagens\\out_rep.mp4'
outlin_path = 'imagens\\out_lin.mp4'
outopt_path = 'imagens\\out_opt.mp4'
outcomb_path = 'imagens\\out_comb.mp4'

outrep = cv2.VideoWriter(outrep_path, fourcc, fps * fator, (outrep_width, outrep_height))
outlin = cv2.VideoWriter(outlin_path, fourcc, fps * fator, (outlin_width, outlin_height))
outopt = cv2.VideoWriter(outopt_path, fourcc, fps * fator, (outopt_width, outopt_height))
outcomb = cv2.VideoWriter(outcomb_path, fourcc, fps * fator, (outcomb_width, outcomb_height))

if not all([outrep.isOpened(), outlin.isOpened(), outopt.isOpened(), outcomb.isOpened()]):
    print("Erro: Não foi possível abrir um ou mais arquivos de saída para escrita.")
    exit()


In [ ]:

def combinar_frames(frames_list):
    """
    Combina uma lista de frames (cada um sendo uma lista de um único frame) horizontalmente.
    Ex: [[frame1], [frame2], [frame3]] -> [frame1 | frame2 | frame3]
    """
    if not frames_list:
        return None
    
    height, width, channels = frames_list[0][0].shape
    
    combined_frame = np.zeros((height, width * len(frames_list), channels), dtype=np.uint8)

    for i, frame_wrapper in enumerate(frames_list):
        combined_frame[:, i * width : (i + 1) * width, :] = frame_wrapper[0]

    return combined_frame


In [ ]:

coord_x, coord_y = np.meshgrid(np.arange(width), np.arange(height))

cont_frames = 0
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
bloco = int(total_frames / 10) if total_frames > 10 else 1

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, prev_frame = cap.read()

if not ret:
    print("Erro: Não foi possível ler o primeiro frame do vídeo.")
    cap.release()
    outrep.release()
    outlin.release()
    outopt.release()
    outcomb.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)


In [ ]:

while True:
    cont_frames += 1

    if cont_frames % bloco == 0:
        print('Processando: ', int(cont_frames / bloco) * 10, '%')
    
    ret, frame = cap.read()

    if not ret:
        break 

    frame_repeat_orig = cv2.resize(prev_frame, (outrep_width, outrep_height))
    frame_linear_orig = cv2.resize(prev_frame, (outlin_width, outlin_height))
    frame_optflow_orig = cv2.resize(prev_frame, (outopt_width, outopt_height))

    frame_combinado_orig = combinar_frames([[frame_repeat_orig], [frame_linear_orig], [frame_optflow_orig]])

    outrep.write(frame_repeat_orig)
    outlin.write(frame_linear_orig)
    outopt.write(frame_optflow_orig)
    outcomb.write(frame_combinado_orig)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    for i in range(1, fator):
        frame_repeat = prev_frame

        frame_linear = cv2.addWeighted(prev_frame, (fator - i) / fator, frame, i / fator, 0)

        map_x = coord_x + (flow[:,:,0] * i / fator)
        map_y = coord_y + (flow[:,:,1] * i / fator)
        
        frame_optflow_warped = cv2.remap(prev_frame, map_x.astype(np.float32), map_y.astype(np.float32), interpolation=cv2.INTER_LINEAR)
        
        peso_fluxo_otico = (fator - i) / fator
        peso_proximo_frame = i / fator
        frame_optflow = cv2.addWeighted(frame_optflow_warped, peso_fluxo_otico, frame, peso_proximo_frame, 0)

        frame_combinado = combinar_frames([[frame_repeat], [frame_linear], [frame_optflow]])
        
        outrep.write(frame_repeat)
        outlin.write(frame_linear)
        outopt.write(frame_optflow)
        outcomb.write(frame_combinado)

    prev_frame = frame
    prev_gray = gray

print("Processamento concluído. Liberando recursos.")
cap.release()
outrep.release()
outlin.release()
outopt.release()
outcomb.release()
print("Vídeos de saída salvos.")

Processando:  10 %
Processando:  20 %
Processando:  30 %
Processando:  40 %
Processando:  50 %
Processando:  60 %
Processando:  70 %
Processando:  80 %
Processando:  90 %
Processando:  100 %
Processamento concluído. Liberando recursos.
Vídeos de saída salvos.


# **Questão 2. Comentários.**

- Para os três métodos de interpolação a onça parece se mexer abruptamente dependendo do tempo de vídeo.
- Estes tempos são os momentos em que há uma diferença muito abrupta entre os frames, ou seja, os momentos em qu eela se mexe muito rapidamente.
- Porém, vale ressaltar que os resultados foram satisfatórios. 
